In [4]:
using Random
using Distributions
using LinearAlgebra
using DataFrames
using XLSX
using StatsBase
using PDMats
using CSV
using Dates
using JuMP
using Ipopt
using NLopt: Opt, objective!, lower_bounds!, upper_bounds!,
               equality_constraint!, xtol_rel!, maxeval!, maxtime!

In [5]:
# Load the CSV
df = CSV.read("Y_full.csv", DataFrame)
select!(df, Not(1))

# Convert to a Matrix{Float64}
dropmissing!(df)
Y_full = Matrix{Float64}(df)

# Omit the lookback period to obtain test data
lookback = 504
Y_test = copy(Y_full[lookback+1:end, :])

2515×10 Matrix{Float64}:
  0.002772     0.00304377   -0.001582     …  -0.00306701  -0.00941998
 -0.0315148   -0.0270803    -0.0225526       -0.0164713   -0.0302804
 -0.035205    -0.0195559    -0.0106637       -0.00201776  -0.00438688
  0.0092706   -0.000122558   0.00724852       0.0102103    0.000777582
  0.0150479    0.0121157     0.0239677        0.0477334    0.0132087
 -0.0223336   -0.0178959    -0.012284     …  -0.00936028  -0.00766867
 -0.00787696  -0.0390295    -0.00544479      -0.0112801   -0.00386406
 -0.00714577  -0.00345951   -0.000746576      0.00507069   0.0100857
 -0.0193923    0.00784288   -0.00485605       0.00252242  -0.00716875
 -0.037105    -0.0224522    -0.00112633       0.010936     0.00489949
  ⋮                                       ⋱               
 -0.00517546  -0.0238215    -0.0109677       -0.0260381   -0.00455082
 -0.0421823   -0.0410992    -0.0196799        0.029247    -0.010888
  0.00440396  -0.0112511     0.0127459       -0.0209517   -0.0100346
  0.0112541

In [6]:
# Dimensions
D   = size(Y_test, 2)
T   = lookback
n_days = size(Y_test, 1)

# Starting portfolio state
x = fill(1/D, D)
h = fill(1000/D, D)
V = [sum(h)]
days_traded = 0

# Transaction costs
kappa = fill(0.001, D)

# Create results CSV
test_csv = "EW.csv"
open(test_csv, "w") do io
    hdr = ["iter", "fees", "V_new"]
    println(io, join(hdr, ","))
end

for i in 1:n_days
    # Rebalance to equal weights
    x_target    = fill(1/D, D)
    V_pre       = sum(h)
    h_proposal  = x_target * V_pre
    fees        = sum(kappa .* abs.(h_proposal .- h))
    V_post      = V_pre - fees

    h .= x_target .* V_post
    x = h ./ sum(h)

    # Apply next‐day returns
    r_next = view(Y_test, 1, :)
    h .= h .* (1 .+ r_next)
    V_new = sum(h)
    x = h ./ V_new

    # Roll returns forward
    Y_test = Y_test[2:end, :]

    # Record & print
    days_traded += 1
    push!(V, V_new)

    row = vcat([days_traded, fees, V_new])
    open(test_csv, "a") do io
        println(io, join(row, ","))
    end
end